# Scraping Menéame

Para crear el dataset de noticias freelance realizamos web scraping en Menéame, en la sección de artículos propios. Al estar configurada en Java Script utilizamos Selenium/webdriver en lugar de BeautifulSoup.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import copy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

In [2]:
CHROMEDRIVER_PATH = r'C:\Users\ladyl\Downloads\chromedriver_win32\chromedriver.exe'
CHROME_PATH = ""
WINDOW_SIZE = "1920,1080"

chrome_options = Options()
# chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH
prefs = {'profile.managed_default_content_settings.images':2}
chrome_options.add_experimental_option("prefs", prefs)

url = "https://www.meneame.net/articles"

last_page = 3

# Hacer lista con todas las url de cada página de noticias 
list_urls = [url] # la inicializamos con la propia home 

for i in range(2, last_page+1): # range 2 porque la 1 es la home
    list_urls.append(url + '?page=' + str(i))

for n, p in enumerate(list_urls):
    
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
        chrome_options=chrome_options
        )

    driver.get(p)

    c = 1

    titulo = []
    texto = []
    url = []
    por = []

    for i in range(1,51):

        try:
            
            # Titular
            try:
                titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').text)
            except: 
                titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').text)

            # URL
            try:
                urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').get_attribute("href")
                url.append(urll)
            except:
                urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').get_attribute("href")
                url.append(urll)

            # Autor
            try:
                por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/div[1]/a[2]').text)
            except:
                por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/div[1]/a[2]').text)

             # Texto
            driver_noticia = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                chrome_options=chrome_options
                )

            driver_noticia.get(urll)

            try:
                texto.append(driver_noticia\
                             .find_element_by_xpath('//*[@id="container"]/div[1]/div/div[1]/div/div').text)
            except:
                texto.append('Revisar')

            driver_noticia.close()

            c = c + 3
        except:
            break
    
    driver.close()


C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options
C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: use options instead of chrome_options


In [3]:
web = ['Menéame'] * len(titulo)
cab = [None] * len(titulo)
com = [None] * len(titulo)
pag = [None] * len(titulo)

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
mn = pd.DataFrame(table_dict)
mn

,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
0,Menéame,Hoy se cumplen 10 años de la noticia más menea...,er11s111,None,None,12389 meneos alcanzó: www.meneame.net/m/dmnm/e...,None,https://www.meneame.net/m/Art%C3%ADculos/hoy-c...
1,Menéame,Tolerancia nula con el alcaide amateur,Ministerio_de_la_Verdad,None,None,Todos lo sabemos : si alguien ocupa tu casa no...,None,https://www.meneame.net/m/Art%C3%ADculos/toler...
2,Menéame,Votaré a Galicia en Común-Anova Mareas el 12 d...,CremaCatalana,None,None,Los voy a votar solamente por las declaracione...,None,https://www.meneame.net/m/Art%C3%ADculos/votar...
3,Menéame,"Venancio, el ciego",Winfield,None,None,Cuando yo era niño vivía en un diseminado (1) ...,None,https://www.meneame.net/m/Art%C3%ADculos/venan...
4,Menéame,Riesgo de muerte,Nagash,None,None,,None,https://www.meneame.net/m/chorradas/riesgo-de-...
5,Menéame,5 mitos del capitalismo,Nagash,None,None,,None,https://www.meneame.net/m/Libertarios/5-mitos-...
6,Menéame,La ultracorrección social,Kernel-chagi,None,None,La ultracorrección es un fenómeno lingüístico ...,None,https://www.meneame.net/m/Art%C3%ADculos/ultra...
7,Menéame,Igual ya no hay final de año,NubisMusic,None,None,Conduciendo por la autovía sobrevino una concl...,None,https://www.meneame.net/m/Art%C3%ADculos/igual...
8,Menéame,Jesús y la resistencia antirromana,Esteban_Rosador,None,None,Que Jesús tuviera una ideología y una praxis a...,None,https://www.meneame.net/m/citas/jesus-resisten...
9,Menéame,MiniPC de escritorio,helisan,None,None,La idea que me había hecho de la Raspberry Pi ...,None,https://www.meneame.net/m/escombrillos/minipc-...


In [4]:
# Exportar a csv
#nombre = 'freelance_' + datetime.today().strftime('%d%m%Y') + '.csv'
#mn.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')

In [4]:
CHROMEDRIVER_PATH = r'C:\Users\ladyl\Downloads\chromedriver_win32\chromedriver.exe'
CHROME_PATH = ""
WINDOW_SIZE = "1920,1080"

chrome_options = Options()
# chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH
prefs = {'profile.managed_default_content_settings.images':2}
chrome_options.add_experimental_option("prefs", prefs)

url = "https://www.meneame.net/articles?page=2"

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
        chrome_options=chrome_options
        )

driver.get(url)

c = 1

titulo = []
texto = []
url = []
por = []

for i in range(1,51):
            
    # Titular
    try:
        titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').text)
    except: 
        titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').text)

    # URL
    try:
        urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').get_attribute("href")
        url.append(urll)
    except:
        urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').get_attribute("href")
        url.append(urll)

    # Autor
    try:
        por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/div[1]/a[2]').text)
    except:
        por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/div[1]/a[2]').text)

    # Texto
    driver_noticia = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
            chrome_options=chrome_options
        )

    driver_noticia.get(urll)

    try:
        texto.append(driver_noticia\
                            .find_element_by_xpath('//*[@id="container"]/div[1]/div/div[1]/div/div').text)
    except:
        texto.append('Revisar')

    driver_noticia.close()

    c = c + 3 

driver.close()

#//*[@id="newswrap"]/div[121]/div/div[2]/h2/a

C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app
C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: use options instead of chrome_options


In [5]:
web = ['Menéame'] * len(titulo)
cab = [None] * len(titulo)
com = [None] * len(titulo)
pag = [None] * len(titulo)

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
mn2 = pd.DataFrame(table_dict)
mn2

,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
0,Menéame,2 noticias se leen mejor juntas: Hacienda me h...,L3FT_BEHIND,None,None,"Solicité dicho subsidio en abril, y en junio e...",None,https://www.meneame.net/m/Art%C3%ADculos/2-not...
1,Menéame,El Libertarianismo y la Libertad Republicana. ...,JohnnyQuest,None,None,Libertad. Es sencillo ¿no? Respeta mi proyecto...,None,https://www.meneame.net/m/Art%C3%ADculos/liber...
2,Menéame,Socialismo para milenials,Nagash,None,None,,None,https://www.meneame.net/m/Libertarios/socialis...
3,Menéame,COVID-19 pone en jaque a la sociabilidad,joseenriquetk,None,None,¿Alguien ha pensado sobre esto?\nNo dejo de pe...,None,https://www.meneame.net/m/Art%C3%ADculos/covid...
4,Menéame,Antes de las velas,Nagash,None,None,,None,https://www.meneame.net/m/chorradas/antes-de-l...
5,Menéame,Guía paso a paso sobre cómo entrenar el modelo...,Tempus_fugit,None,None,El modelo Schopenhauer-GPT-2 fue entrenado usa...,None,https://www.meneame.net/m/ciencia/guia-paso-pa...
6,Menéame,"A propósito de ""Filólogos cabreados""",Orégano,None,None,A propósito de Filólogos cabreados y la elimin...,None,https://www.meneame.net/m/ciencia/proposito-fi...
7,Menéame,¿Por qué suben los precios? La verdad sobre la...,Nagash,None,None,,None,https://www.meneame.net/m/Libertarios/suben-pr...
8,Menéame,Argentina : La mitad del gasto público se fina...,Nagash,None,None,,None,https://www.meneame.net/m/Libertarios/argentin...
9,Menéame,La mascarilla y la foto,Feindesland,None,None,No creo que valga la pena retomar la discusión...,None,https://www.meneame.net/m/Art%C3%ADculos/masca...


In [6]:
CHROMEDRIVER_PATH = r'C:\Users\ladyl\Downloads\chromedriver_win32\chromedriver.exe'
CHROME_PATH = ""
WINDOW_SIZE = "1920,1080"

chrome_options = Options()
# chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH
prefs = {'profile.managed_default_content_settings.images':2}
chrome_options.add_experimental_option("prefs", prefs)

url = "https://www.meneame.net/articles?page=3"

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
        chrome_options=chrome_options
        )

driver.get(url)

c = 1

titulo = []
texto = []
url = []
por = []

for i in range(1,51):
    try:        
        # Titular
        try:
            titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').text)
        except: 
            titulo.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').text)

        # URL
        try:
            urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/h2/a').get_attribute("href")
            url.append(urll)
        except:
            urll = driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/h2/a').get_attribute("href")
            url.append(urll)

        # Autor
        try:
            por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[2]/div[1]/a[2]').text)
        except:
            por.append(driver.find_element_by_xpath('//*[@id="newswrap"]/div[' + str(c) + ']/div/div[3]/div[1]/a[2]').text)

        # Texto
        driver_noticia = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                chrome_options=chrome_options
            )

        driver_noticia.get(urll)

        try:
            texto.append(driver_noticia\
                                .find_element_by_xpath('//*[@id="container"]/div[1]/div/div[1]/div/div').text)
        except:
            texto.append('Revisar')

        driver_noticia.close()

        c = c + 3 
    except:
        break

driver.close()


C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app
C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: use options instead of chrome_options


In [7]:
web = ['Menéame'] * len(titulo)
cab = [None] * len(titulo)
com = [None] * len(titulo)
pag = [None] * len(titulo)

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
mn3 = pd.DataFrame(table_dict)
mn3

,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
0,Menéame,Hoy se cumplen 10 años de la noticia más menea...,er11s111,None,None,12389 meneos alcanzó: www.meneame.net/m/dmnm/e...,None,https://www.meneame.net/m/Art%C3%ADculos/hoy-c...
1,Menéame,Tolerancia nula con el alcaide amateur,Ministerio_de_la_Verdad,None,None,Todos lo sabemos : si alguien ocupa tu casa no...,None,https://www.meneame.net/m/Art%C3%ADculos/toler...
2,Menéame,Votaré a Galicia en Común-Anova Mareas el 12 d...,CremaCatalana,None,None,Los voy a votar solamente por las declaracione...,None,https://www.meneame.net/m/Art%C3%ADculos/votar...
3,Menéame,"Venancio, el ciego",Winfield,None,None,Cuando yo era niño vivía en un diseminado (1) ...,None,https://www.meneame.net/m/Art%C3%ADculos/venan...
4,Menéame,Riesgo de muerte,Nagash,None,None,,None,https://www.meneame.net/m/chorradas/riesgo-de-...
5,Menéame,5 mitos del capitalismo,Nagash,None,None,,None,https://www.meneame.net/m/Libertarios/5-mitos-...
6,Menéame,La ultracorrección social,Kernel-chagi,None,None,La ultracorrección es un fenómeno lingüístico ...,None,https://www.meneame.net/m/Art%C3%ADculos/ultra...
7,Menéame,Igual ya no hay final de año,NubisMusic,None,None,Conduciendo por la autovía sobrevino una concl...,None,https://www.meneame.net/m/Art%C3%ADculos/igual...
8,Menéame,Jesús y la resistencia antirromana,Esteban_Rosador,None,None,Que Jesús tuviera una ideología y una praxis a...,None,https://www.meneame.net/m/citas/jesus-resisten...
9,Menéame,MiniPC de escritorio,helisan,None,None,La idea que me había hecho de la Raspberry Pi ...,None,https://www.meneame.net/m/escombrillos/minipc-...


## Dataframe conjunto

In [8]:
dfs = [mn, mn2, mn3]

# Los unimos, y para que no se repitan los índices, los reseteamos
df = pd.concat(dfs).reset_index(drop=True)

# Para crear el id (único) de cada noticia
new_id = 'new_' + (df.index + 1).astype(str)

# Y lo insertamos en la primera posición
df.insert(loc=0, column='new_id', value=new_id)

# Ver el dataframe final
df

,new_id,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
0,new_1,Menéame,Hoy se cumplen 10 años de la noticia más menea...,er11s111,None,None,12389 meneos alcanzó: www.meneame.net/m/dmnm/e...,None,https://www.meneame.net/m/Art%C3%ADculos/hoy-c...
1,new_2,Menéame,Tolerancia nula con el alcaide amateur,Ministerio_de_la_Verdad,None,None,Todos lo sabemos : si alguien ocupa tu casa no...,None,https://www.meneame.net/m/Art%C3%ADculos/toler...
2,new_3,Menéame,Votaré a Galicia en Común-Anova Mareas el 12 d...,CremaCatalana,None,None,Los voy a votar solamente por las declaracione...,None,https://www.meneame.net/m/Art%C3%ADculos/votar...
3,new_4,Menéame,"Venancio, el ciego",Winfield,None,None,Cuando yo era niño vivía en un diseminado (1) ...,None,https://www.meneame.net/m/Art%C3%ADculos/venan...
4,new_5,Menéame,Riesgo de muerte,Nagash,None,None,,None,https://www.meneame.net/m/chorradas/riesgo-de-...
...,...,...,...,...,...,...,...,...,...
143,new_144,Menéame,El nuevo rescate bancario que está ocurriendo ...,gusy,None,None,¿Cómo evitamos llegar a 10 millones de parados...,None,https://www.meneame.net/m/Art%C3%ADculos/nuevo...
144,new_145,Menéame,"Gatos, perros, lagartos...",ContinuumST,None,None,Cuatro gatos llegaron el día seis a las ocho d...,None,https://www.meneame.net/m/relatocorto/gatos-pe...
145,new_146,Menéame,Fábula del reloj del padre,Ferran,None,None,"Antes de morir, un padre le dijo a su hijo: “E...",None,https://www.meneame.net/m/Art%C3%ADculos/fabul...
146,new_147,Menéame,Una muestra de buena educación,Feindesland,None,None,Que te inviten a la boda de tu novia es una mu...,None,https://www.meneame.net/m/relatocorto/muestra-...


In [12]:
df.sample(5)

,new_id,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
89,new_90,Menéame,Pesadilla con operadora,offler,None,None,La de gente que habrá con problemas con su ope...,None,https://www.meneame.net/m/Art%C3%ADculos/pesad...
52,new_53,Menéame,COVID-19 pone en jaque a la sociabilidad,joseenriquetk,None,None,¿Alguien ha pensado sobre esto?\nNo dejo de pe...,None,https://www.meneame.net/m/Art%C3%ADculos/covid...
36,new_37,Menéame,Las reglas de oro para una comedia de éxito,ContinuumST,None,None,Una chica bonita es mejor que una fea.\nUna pi...,None,https://www.meneame.net/m/Fragmentos/reglas-or...
123,new_124,Menéame,Sars-CoV-2 y PCR,pablisako,None,None,"A lo largo del desarrollo de la pandemia, han ...",None,https://www.meneame.net/m/ciencia/sars-cov-2-y...
60,new_61,Menéame,¿Qué es el Sexo? Obra de Lynn Margulis y Dorio...,Tempus_fugit,None,None,Acabo de terminar de leer uno de los libros má...,None,https://www.meneame.net/m/ciencia/sexo-obra-ly...


In [13]:
# Exportar a csv
nombre = 'freelance_' + datetime.today().strftime('%d%m%Y') + '.csv'
df.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')